# Batch Lamda1050 jump correction script and smoothing

### Author: Vinicio Soto, CICIMA
### This script works with raw Lambdas1050's ASC files in batch. It corrects Lambdas1050's detector jump between 857 nm and 858 nm and performs a Savitzky-Golay filtering on the corrected data

#### exclusion_list.txt: You can add a file with this name to the folder with the data to exclude any file with that name

This script reads every l1050 file in folder and its subfolders and create a unique average, jump_corrected_files, report and std_dev folders
It creates sections for each folder with its information


### Dependencies

In [1]:
import pandas as pd #Data analysis
import numpy as np  #Array and numeric methods
from matplotlib.backends.backend_pdf import PdfPages #Print PDFS
import matplotlib #Print graphs
import matplotlib.pyplot as plt #Print graphs
import os #operating system
import re #regular expression manipulation
from datetime import datetime #date and time methods

#import spectraltools
#This script requires the file spectraltools.py to work
import sys
# Add the external folder to the system path
current_dir = os.getcwd()
external_folder_path = os.path.abspath(os.path.join(current_dir, '../libraries'))
sys.path.append(external_folder_path)


#This line of code allow us to access data in colab
#functionality to reload modules

import importlib
import spectraltools
import metrics
import datapath_selector

# clear the import cache
importlib.reload(metrics)
importlib.reload(spectraltools)
importlib.reload(datapath_selector)
# now you can import my_class and it'll be updated
from metrics import *
from spectraltools import *
from datapath_selector import get_paths
from pathlib import Path




### Instructions:

Write your archive folders' path in file_folder_data_path list

In [2]:
#parent folder path

#file_folder_data_path =[ #r"C:\Users\EstebanSoto\Jupyter\escarabajos\L1050_data\CICIMA-2024-05-REFLECTANCE\DORSAL",
                         #r"C:\Users\EstebanSoto\Jupyter\escarabajos\L1050_data\CICIMA-2024-05-REFLECTANCE\VENTRAL",
                         #r"C:\Users\EstebanSoto\Jupyter\escarabajos\L1050_data\CICIMA-2024-03-REFLECTANCE\without iris nor lens",
                         #r"C:\Users\EstebanSoto\Jupyter\escarabajos\L1050_data\2024-04-INBUCR-REFLECTANCE",
                         #r"C:\Users\EstebanSoto\Jupyter\escarabajos\L1050_data\2023-03-CICIMAUCR-2-REFLECTANCE",
                         #r"C:\Users\esteb\cicima\escarabajos\L1050_data\CICIMA-2024-05-REFLECTANCE\DORSAL"
                       #]
parent_folder_data_path = Path(r"C:\Users\esteb\cicima\escarabajos\test\data\test_spectraltools\jump_correction_CRAIC") 
report_path = (parent_folder_data_path.parent / "reports" ) / parent_folder_data_path.name
print(report_path)

C:\Users\esteb\cicima\escarabajos\test\data\test_spectraltools\reports\jump_correction_CRAIC


#### Collections

In [3]:
"""OPTIONS: cicima_laptop, colaboratory, wfh, cicima_desktop
    """
collection_paths = get_paths()
print(collection_paths)

angsol_collection = Specimen_Collection("ANGSOL", collection_paths["angsol_collection_path"] , collection_paths["angsol_collection_metadata"] , "HIGH")
angsol_collection.set_description("ANGSOL collection has specimens that belong to Angel Solís. The confidence that we have about specimen identification is high.")

cicimaucr_collection = Specimen_Collection("CICIMAUCR1", collection_paths["cicimaucr_collection_path"] , collection_paths["cicima_ucr_metadata"] , "HIGH")
cicimaucr_collection_2 = Specimen_Collection("CICIMAUCR2", collection_paths["cicimaucr_collection_2_path"] , collection_paths["cicima_ucr_metadata"] , "HIGH")
cicimaucr_collection_3 = Specimen_Collection("CICIMAUCR3", collection_paths["cicimaucr_collection_3_path"] , collection_paths["cicima_ucr_metadata"] , "HIGH")
inbucr_collection = Specimen_Collection("INBUCR", collection_paths["inbucr_collection_path"] , collection_paths["inbucr_metadata"] , "HIGH")
bioucr_collection = Specimen_Collection("BIOUCR", collection_paths["bioucr_collection_path"] , collection_paths["bioucr_metadata"] , "LOW")

collection_list = [
                    angsol_collection,
                    cicimaucr_collection,
                    cicimaucr_collection_2,
                    cicimaucr_collection_3,
                    inbucr_collection,
                    bioucr_collection,
                    ]
print(collection_list)

{'angsol_collection_path': WindowsPath('C:/Users/esteb/cicima/escarabajos/L1050_data/ANGSOL/average'), 'angsol_collection_metadata': WindowsPath('C:/Users/esteb/cicima/escarabajos/L1050_data/collections/CICIMA-beetles-general-inventory - ANGSOL.txt'), 'cicimaucr_collection_path': WindowsPath('C:/Users/esteb/cicima/escarabajos/L1050_data/TRA_data_CICIMA_INBUCR/CICIMAUCR/reflectance'), 'cicimaucr_collection_2_path': WindowsPath('C:/Users/esteb/cicima/escarabajos/L1050_data/CICIMA-2024-01-REFLECTANCE/average'), 'cicimaucr_collection_3_path': WindowsPath('C:/Users/esteb/cicima/escarabajos/L1050_data/CICIMA-2024-03-REFLECTANCE/without iris nor lens/average'), 'cicimaucr_collection_4_path': WindowsPath('C:/Users/esteb/cicima/escarabajos/L1050_data/CICIMA-2024-05-REFLECTANCE/DORSAL/average'), 'cicima_ucr_metadata': WindowsPath('C:/Users/esteb/cicima/escarabajos/L1050_data/collections/CICIMA-beetles-general-inventory - CICIMAUCR.txt'), 'inbucr_collection_path': WindowsPath('C:/Users/esteb/cici

### Constants
Define your constants

In [4]:
#determine 
markersize = 1
#date
date = str(datetime.today().date())

### Functions

#### Check if it is a L1050 file or a CRAIC file

#### Get genera and species function

### Correction process function

In [5]:
def jump_correction(Spectrum):
    """Receives a l1050 spectrum and corrects its detector jump"""
    metadata = Spectrum.get_metadata()
    df = Spectrum.get_dataframe()
    
    #operating_mode
    operating_mode = metadata["operating_mode"]
    
    #convert columns to float
    df['wavelength'] = df['wavelength'].astype(float)
    df[operating_mode] = df[operating_mode].astype(float)
    

    #This code finds the rows before and after the detector change
    #freq1 is the frequency before the change and freq2 is the frequency after the change
    wavelenght1 = 857.000000 #857
    wavelenght2 = 858.000000 #858
    
    #print("Loc")
    row1 = df.loc[ df['wavelength'] == wavelenght1]
    row2 = df.loc[ df['wavelength'] == wavelenght2]
    row12 = df.loc[ df['wavelength'] == wavelenght1-1]
    row13 = df.loc[ df['wavelength'] == wavelenght1-2]
    row14 = df.loc[ df['wavelength'] == wavelenght1-3]
    row15 = df.loc[ df['wavelength'] == wavelenght1-4]
    
    #This code finds the values corresponding to those frequencies and creates a correction factor
    
    try:
        data1 = row1.iat[0,1]
        data2 = row2.iat[0,1]
        data22 = row12.iat[0,1]
        data23 = row13.iat[0,1]
        data24 = row14.iat[0,1]
        data25 = row15.iat[0,1]
    
        data1_avg = (data1 + data22+data23)/3 #+data24+data25)/5
        data2_avg = (data2)
        correction_factor= data1_avg/data2_avg  
    
        #Multiply all frequencies equal or greater than freq2 by correction_factor
        df2 = df
        df2.loc[df2["wavelength"] >= wavelenght2 , [operating_mode]] *= correction_factor
        
        
        return df
    except Exception as e:
        
        print(e)
        return pd.DataFrame([])
    
    

In [6]:
def save_corrected_df(df, filename,metadata):
    #variables
    header = metadata["header"]
    
    #convert dataframe to numpy
    my_numpy_df_data = df.to_numpy()
    
    #Create new folder
    corrected_path = filename

    if not os.path.exists(corrected_path):
        os.mkdir(corrected_path)

    #saves file, removes .ASC and removes .SAMPLE
    unnecessary_text = [".Sample", ".Cycle1"]
    temp_text = filename.replace('.ASC', ''+'.txt')
    
    for text in unnecessary_text:
        #print(temp_text)
        temp_text = temp_text.replace(text, "")

    #new_archive_name = corrected_path + temp_text
    new_archive_name = os.path.join(corrected_path , temp_text)

    with open(new_archive_name, 'wb') as f:
        np.savetxt(new_archive_name, my_numpy_df_data,fmt = "%.6f", delimiter="\t", header=header, comments='', )
    with open(new_archive_name, "r") as f:
        data = f.read()
    with open(new_archive_name, "w") as f:
        f.write(data.replace("#DATA\n","#DATA"))

### Define Savitzky-Golay Filter

In [7]:
from scipy.signal import savgol_filter

def savitzky_golay_filter(metadata, df) -> pd.DataFrame:
    """Defines a Savitzky Golay Filter: Basically, smooths out the function. https://es.wikipedia.org/wiki/Filtro_de_Savitzky%E2%80%93Golay"""
    operating_mode = metadata["operating_mode"]
    y = df[operating_mode] = df[operating_mode].astype(float)
    
    # Apply Savitzky-Golay filter
    window_length = 21  # Window length (odd number)
    polyorder = 2  # Polynomial order
    y_smooth = savgol_filter(y, window_length, polyorder)
    
    #create new dataframe
    df_smooth = pd.DataFrame([])
    df_smooth["wavelength"] =df["wavelength"]
    df_smooth[operating_mode] = y_smooth
    
    return df_smooth
    

### Define function that filters selected extensions

In [8]:
def list_all_filepaths(parent_folder):
    filepaths = []
    
    # Walk through the directory
    for root, dirs, files in os.walk(parent_folder):
        for file in files:
            # Combine the root directory with the file name to get the full path
            filepath = os.path.join(root, file)
            filepaths.append(filepath)
    
    return filepaths
file_list = list_all_filepaths(parent_folder_data_path)
print(f"{file_list=}")

file_list=['C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_L.csv', 'C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_O.csv', 'C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\average\\BIOUCR0030.txt', 'C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\std_dev\\BIOUCR0030.txt']


In [9]:
def get_exclusion_list(file_list):
        """Only files whose name is in the exclusion list"""
        #file extension
        file_extension = "exclusion_list"
    
        # filters a list of strings to create a new list containing only the elements that have exclusion_list"
    
        def filter_substring_elements(path_strings, substring):
            filtered_paths = [path for path in path_strings if substring in path]
            return filtered_paths
    
        # Filter elements
        filtered_list = filter_substring_elements(file_list, file_extension)
    
        # Displaying the filtered list
        #print(filtered_list)
        
        return filtered_list
exclusion_list = get_exclusion_list(file_list)
print(f"{exclusion_list=}")

exclusion_list=[]


In [10]:
def filter_list_extension(file_list):
    """Only files that end in a particular extension are read"""
    #file extension
    file_extensions = [".ASC", ".csv"]

    filtered_list = []
    # filters a list of strings to create a new list containing only the elements that end with ".ASC"

    def filter_substring_elements(path_strings, substring):
            filtered_paths = [path for path in path_strings if substring in path]
            return filtered_paths
        
    for extension in file_extensions:
        # Filtering elements ending with the extension
        filtered_list += filter_substring_elements(file_list, extension)

    # Displaying the filtered list
    #print(filtered_list)
    
    return filtered_list
filtered_list = filter_list_extension(file_list)

print(f"{filtered_list=}")

filtered_list=['C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_L.csv', 'C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_O.csv']


In [11]:
def filter_failed_runs(filtered_list):
    """Filter runs that failed"""
    #file extension
    fail_strings = ["fail"]

    # filters a list of strings to create a new list containing only the elements that end with ".ASC"

    def filter_substring_elements(path_strings, substring):
        filtered_paths = [path for path in path_strings if substring not in path]
        return filtered_paths

    # Filtering elements with any of the fail strings
    for string in fail_strings: 
        filtered_list = filter_substring_elements(filtered_list, string)

    # Displaying the filtered list
    #print(filtered_list)
    
    return filtered_list

successful_list = filter_failed_runs(filtered_list)
print(f"{successful_list=}")

successful_list=['C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_L.csv', 'C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_O.csv']


### Logic: Remove jumps

In [12]:
### First, we will get a list of filenames in the folder
filenames = []
for path in successful_list: 
    filenames.append(os.path.basename(path))

# From this list we will make a set of codes.
#print(filenames)

def get_sample_code(row_str):
        #Tries to get the sample code from the filename
        re1 = r"([a-zA-Z\d]+)(?:-\d)*(?:.Sample)*.(?:txt)*(?:ASC)*"
        
        #Names are in the form CODE-MEASUREMENTNUMBER.TXT
        p = re.compile(re1)
        m = p.match(row_str)

        if m:
            return(m.group(1))
        else:
            ""
codes = []
for filename in filenames:
    codes.append(get_sample_code(filename))
codes = set(codes)
codes = sorted(codes)

print(f"codes {codes}")

codes ['BIOUCR0030']


In [13]:
#for each file in the exclusion list
exclusion_list_df = pd.DataFrame()
for file in exclusion_list:
    df = pd.read_csv(file, header = 0, names=["filename", "reason"], delimiter = "\t")
    exclusion_list_df = pd.concat([exclusion_list_df, df], ignore_index = True)
print(exclusion_list_df)
print(f"{exclusion_list_df=}")

Empty DataFrame
Columns: []
Index: []
exclusion_list_df=Empty DataFrame
Columns: []
Index: []


In [14]:
#remove excluded files
def remove_excluded_files(successful_list, exclusion_list_df):
    filtered_list = []

    #if the file is not empty it gets all files that are not in that list
    if not exclusion_list_df.empty:
        #gets a exclusion list
        exclusion_list = exclusion_list_df["filename"].values
        for excluded_file in exclusion_list:
                    filtered_list = [filename for filename in successful_list if excluded_file not in filename ]
    filtered_list = successful_list
    return filtered_list
    
successful_list_without_excluded_files = remove_excluded_files(successful_list, exclusion_list_df)
print(f"{successful_list_without_excluded_files=}")

successful_list_without_excluded_files=['C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_L.csv', 'C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_O.csv']


In [21]:
def polarization_sum(files_with_code):
    #get every craic file in files with code
    craic_files = [file for file in files_with_code if check_CRAIC_file(file)]
    print(f"{craic_files=}")
    #get the collection for these files
    craic_file_collection = []
    for file in craic_files:
        craic_file_collection.append(Specimen_Collection.collection_lookup(file, collection_list))
    #get the spectra for these files
    craic_spectra =[]
    for file, collection in zip(craic_files, craic_file_collection):
        craic_spectra.append(Spectrum(file,collection))
    #if any of these spectrums is not polarized return that spectrum
    non_polarized_spectrum = [spectrum for spectrum in craic_spectra if spectrum.get_polarization() == "O"]
    if len(non_polarized_spectrum) != 0:
        return non_polarized_spectrum[0].get_dataframe()
     
    #if not check if there is a left and right spectrum and sum it
    else:
        right_spectrum = [spectrum for spectrum in craic_spectra if spectrum.get_polarization() == "R"]
        left_spectrum = [spectrum for spectrum in craic_spectra if spectrum.get_polarization() == "L"]

        def sum_spectra(right_spectrum, left_spectrum):
            df = pd.DataFrame()
            df_r = right_spectrum.get_dataframe()
            df_l = right_spectrum.get_dataframe()

            # Sum %R for each wavelength
            sum_r = df_r.groupby('wavelength')['%R'].sum().reset_index()
            sum_l = df_l.groupby('wavelength')['%R'].sum().reset_index()

            # Merge the results
            merged = pd.merge(sum_r, sum_l, on='wavelength', suffixes=('_r', '_l'))
            
            # Calculate the total %R and normalize
            df['total_%R'] = merged['%R_r'] + merged['%R_l']
            max_total = df['total_%R'].max()
            df['%R'] = (merged['total_%R'] / max_total) * 100

            #drop total_%R
            df = df.drop("total_%R")
            
            return df
        if (right_spectrum and left_spectrum):
            return sum_spectra(right_spectrum, left_spectrum)
        if (not right_spectrum) and (not left_spectrum) :
            warnings.warn(f"Right and left spectrum missing:", UserWarning)
        if (not right_spectrum):
            warnings.warn(f"Right spectrum missing:", UserWarning)
        if (not left_spectrum) :
            warnings.warn(f"Left spectrum missing:", UserWarning)

In [22]:
#Create report
def create_report(filtered_list):
    """This function creates a report"""
    jump_corrected_dict = []
    file_counter = 0
    
    #create a subfolder called report with the correction process info  
    report_path.mkdir(parents=True, exist_ok=True)

    #create report name
    new_file = os.path.join( report_path , f'original_data_{date}.pdf')

    #make a list of all files with code
    files_with_code = []
    for code in codes:
        files_with_code += [file for file in filtered_list if code in file]

    print(f"{files_with_code=}")
    
    #create a report in pdf
    with matplotlib.backends.backend_pdf.PdfPages(new_file) as pdf:
        
        for code in codes: 
            #Code by code we will plot all the spectra with the same name
            code_file_counter = 1
            
            files_with_code = [file for file in filtered_list if code in file]
            
            plt.figure(figsize=(10, 4))
            
            for file in files_with_code:
                
                #print(f"{code_file_counter}")
                
                #create a spectrum object
                collection = Specimen_Collection.collection_lookup(code, collection_list)
                spectrum = Spectrum(file, collection)
                
                metadata, df = spectrum.get_metadata(), spectrum.get_dataframe()
                filename = spectrum.get_filename()
                basename = Path(filename).name
                
                #create figure with every contribution
                color = "r"
                if code_file_counter == 2:
                    color = "b"
                elif code_file_counter ==3:
                    color = "g"
                elif code_file_counter ==4:
                    color = "cyan"
                elif code_file_counter ==5:
                    color = "magenta"
                elif code_file_counter ==6:
                    color = "yellow"
                elif code_file_counter ==7:
                    color = "black"
            
                
                #plots before changes
                plt.plot(df["wavelength"], df[metadata["operating_mode"]], color = color, label = f"{basename}")
                
                #If the file is a l1050 file it will be jump corrected
                #Fix jump correction
                if check_l1050_file(file):
                    df_corrected = jump_correction(spectrum)
                elif check_CRAIC_file(file): 
                    #If the file is a CRAIC file nothing must be done
                    df_corrected = spectrum.get_dataframe()
                else:
                    df_corrected = spectrum.get_dataframe()
                    
                corrected_dict = {"df":df_corrected, "complete_path": file, "metadata": metadata, "code":code}
                
                jump_corrected_dict.append(corrected_dict)
                save_corrected_df(df_corrected, filename, metadata)
                
                code_file_counter += 1
                file_counter += 1
                ### end for
                
            plt.title(f'1. Raw data. {code}')
            plt.ylim(0,100)
            plt.legend()
            plt.grid(True)
            #plt.show()
            pdf.savefig()
            plt.close()

create_report(successful_list_without_excluded_files)

files_with_code=['C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_L.csv', 'C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_O.csv']
codes={'ANGSOL0021', 'ANGSOL0029', 'ANGSOL0035', 'ANGSOL0019', 'ANGSOL0011', 'ANGSOL0002', 'ANGSOL0016', 'ANGSOL0028', 'ANGSOL0031', 'ANGSOL0022', 'ANGSOL0033', 'ANGSOL0027', 'ANGSOL0010', 'ANGSOL0013', 'ANGSOL0030', 'ANGSOL0020', 'ANGSOL0015', 'ANGSOL0018', 'ANGSOL0003', 'ANGSOL0017', 'ANGSOL0007', 'ANGSOL0024', 'ANGSOL0026', 'ANGSOL0005', 'ANGSOL0004', 'ANGSOL0034', 'ANGSOL0032', 'ANGSOL0001', 'ANGSOL0023', 'ANGSOL0008', 'ANGSOL0009', 'ANGSOL0012', 'ANGSOL0006', 'ANGSOL0014', 'ANGSOL0025'}
first_line='Time1=43ms:Average1=10:Objective=10X:Aperture=1: (3/5/2024 8:54:50 AM)\n'
first_line='Time1=43ms:Average1=10:Objective=10X:Aperture=1: (3/5/2024 8:54:50 AM)\n'
CRAIC file
File: C:\Users\esteb\cicima\escarabajos\test\data\test_spectraltools\jump_c

C:\Users\esteb\cicima\escarabajos\libraries\spectraltools.py:109: UserWarning: The provided code (BIOUCR0030) is not in the collection list:
  warnings.warn(f"The provided code ({code}) is not in the collection list:", UserWarning)
C:\Users\esteb\cicima\escarabajos\libraries\spectraltools.py:109: UserWarning: The provided code (C:\Users\esteb\cicima\escarabajos\test\data\test_spectraltools\jump_correction_CRAIC\BIOUCR0030_L.csv) is not in the collection list:
  warnings.warn(f"The provided code ({code}) is not in the collection list:", UserWarning)
C:\Users\esteb\cicima\escarabajos\libraries\spectraltools.py:109: UserWarning: The provided code (C:\Users\esteb\cicima\escarabajos\test\data\test_spectraltools\jump_correction_CRAIC\BIOUCR0030_O.csv) is not in the collection list:
  warnings.warn(f"The provided code ({code}) is not in the collection list:", UserWarning)
C:\Users\esteb\cicima\escarabajos\libraries\spectraltools.py:109: UserWarning: The provided code (BIOUCR0030) is not in th

#calculate averages

In [23]:
def get_files_with_same_name(filtered_list):
    #get files in directory
    files = []
    for file in filtered_list:
        files.append( os.path.basename(file))
    
    #print(files)
    
    sample_names = []
    re1 = r"([a-zA-Z\d]+)(?:-\d)*(?:.Sample)*.(?:txt)*(?:ASC)*"
    #Names are in the form CODE-MEASUREMENTNUMBER.TXT
    for file in files:
        #print(file)
        p = re.compile(re1)
        m = p.match(file)
        if m:
            sample_names.append(m.group(1))

    return set(sample_names)

In [24]:
var_dataframes = None
def average_files(file_list, code_set, directory, pdf):

    
    #parent directory
    jump_corrected_path = os.path.join( directory , "jump_corrected_files")
    save_directory = os.path.dirname(jump_corrected_path)
    
    #print(f"{save_directory=}")
    file_counter = 0

    #Look for every available code in the folder
    for code in code_set:
        print(f"{code=}") 
        
        dataframes = []
        # Read each file, create dataframes, and plot them
        for file_path in file_list:
            print(f"{file_path=}")

            #get metadata and dataframe
            print("Reading file...")
            print("Getting metadata and dataframe...")
            collection = Specimen_Collection.collection_lookup(code, collection_list)
            spectrum = Spectrum(file_path, collection)
            metadata, df = spectrum.get_metadata(), spectrum.get_dataframe()
            print(f"{df=}")
            #metadata info
            operating_mode = metadata["operating_mode"]
            sample_name = metadata["code"] 

            # Read the file into a pandas DataFrame
            #convert columns to float
            print("Converting columns to float...")
            df['wavelength'] = df['wavelength'].astype(float)
            df[operating_mode] = df[operating_mode].astype(float)
            
            print("Dropping index...")
            try:
                df = df.drop("index", axis = 1)
            except Exception as e:
                print(e)
            print("Printing head...")
            print(df.head())
            # Append the dataframe to the list if not empty
            if not df.empty and not None and len(df) != 0:
                dataframes.append(df) 
            
        # Calculate the average of all dataframes ignore empty ones
        # average l1050 files and polarization_sum CRAIC files
        print("elements")
        
        #print(dataframes)
        print(len(dataframes))

        #If there are no dataframes, continue
        if len(dataframes) == 0:
            continue
        #If there is only one 
        elif len(dataframes) == 1:
            average_df = dataframes[0]
            std_dev_df = pd.DataFrame([])
        #If there are many
        else:
            average_df = pd.concat(dataframes).groupby('wavelength').mean().reset_index()
            std_dev_df = pd.concat(dataframes).groupby('wavelength').std().reset_index()

        try: 
            #print(f"{average_df=}")
            
            #average plot
            avg_plot = average_df.plot(x='wavelength', y=operating_mode, kind='scatter', s=markersize, title = sample_name + " avg", figsize = (10, 4), grid=True)
            #save figures
            pdf.savefig()
            #close plot
            plt.close()
            
            #standard deviation plot
            std_plot = std_dev_df.plot(x='wavelength', y=operating_mode, color = "r", kind='scatter', s=markersize, title = sample_name + " std dev", figsize = (10,4), grid=True)
            #save figures
            pdf.savefig()
            #close plot
            plt.close()
        except Exception as e:
            print(e)
            
        #SAVE FILE
        #Convert to numpy
        average_np = average_df.to_numpy()
        std_dev_np = std_dev_df.to_numpy()
        
        #Create new folder
      
        avg_new_path = os.path.join(save_directory , "average")
        std_dev_new_path = os.path.join(save_directory , "std_dev") 

        
        if not os.path.exists(avg_new_path):
            os.mkdir(avg_new_path)
        if not os.path.exists(std_dev_new_path):
            os.mkdir(std_dev_new_path)

        #creates new filename
        #print(f"sample name: {sample_name}")

        new_avg_archive_name = os.path.join(avg_new_path, sample_name+'.txt')
        new_std_dev_archive_name = os.path.join( std_dev_new_path , sample_name+'.txt')

        #print(new_avg_archive_name)
        #saves files
        np.savetxt(new_avg_archive_name, average_np,fmt = "%.6f", delimiter="\t", header=metadata["header"], comments='')
        np.savetxt(new_std_dev_archive_name, std_dev_np,fmt = "%.6f", delimiter="\t", header=metadata["header"], comments='')
        #print("fin de la iteracion")

        def delete_blank_row(archive_name):
            with open(archive_name, "r") as f:
                data = f.read()
            with open(archive_name, "w") as f:
                f.write(data.replace("#DATA\n","#DATA"))

        delete_blank_row(new_avg_archive_name)
        delete_blank_row(new_std_dev_archive_name)
                         
        file_counter = file_counter +1
        
    print(f"{file_counter} averages were calculated")
        
        

In [25]:
### Calculate the average for each spectrum if it has the same name
### And create a p
avg_std_report_file = report_path.parent / "average_and_std_report.pdf"
#print(avg_std_report_file)

with matplotlib.backends.backend_pdf.PdfPages(avg_std_report_file) as pdf:
    code_set = get_files_with_same_name(successful_list_without_excluded_files)
    print(code_set)
    average_files(successful_list_without_excluded_files, code_set, parent_folder_data_path , pdf)

{'BIOUCR0030'}
code='BIOUCR0030'
file_path='C:\\Users\\esteb\\cicima\\escarabajos\\test\\data\\test_spectraltools\\jump_correction_CRAIC\\BIOUCR0030_L.csv'
Reading file...
Getting metadata and dataframe...
codes={'ANGSOL0021', 'ANGSOL0029', 'ANGSOL0035', 'ANGSOL0019', 'ANGSOL0011', 'ANGSOL0002', 'ANGSOL0016', 'ANGSOL0028', 'ANGSOL0031', 'ANGSOL0022', 'ANGSOL0033', 'ANGSOL0027', 'ANGSOL0010', 'ANGSOL0013', 'ANGSOL0030', 'ANGSOL0020', 'ANGSOL0015', 'ANGSOL0018', 'ANGSOL0003', 'ANGSOL0017', 'ANGSOL0007', 'ANGSOL0024', 'ANGSOL0026', 'ANGSOL0005', 'ANGSOL0004', 'ANGSOL0034', 'ANGSOL0032', 'ANGSOL0001', 'ANGSOL0023', 'ANGSOL0008', 'ANGSOL0009', 'ANGSOL0012', 'ANGSOL0006', 'ANGSOL0014', 'ANGSOL0025'}
first_line='Time1=43ms:Average1=10:Objective=10X:Aperture=1: (3/5/2024 8:54:50 AM)\n'
first_line='Time1=43ms:Average1=10:Objective=10X:Aperture=1: (3/5/2024 8:54:50 AM)\n'
CRAIC file
File: C:\Users\esteb\cicima\escarabajos\test\data\test_spectraltools\jump_correction_CRAIC\BIOUCR0030_L.csv
row_st

C:\Users\esteb\cicima\escarabajos\libraries\spectraltools.py:109: UserWarning: The provided code (BIOUCR0030) is not in the collection list:
  warnings.warn(f"The provided code ({code}) is not in the collection list:", UserWarning)
C:\Users\esteb\cicima\escarabajos\libraries\spectraltools.py:109: UserWarning: The provided code (BIOUCR0030) is not in the collection list:
  warnings.warn(f"The provided code ({code}) is not in the collection list:", UserWarning)


1 averages were calculated


#Create jump_corrected_files report

avg_std_report_file = os.path.join(report_path, "detector_jump_correction_report.pdf")
print(avg_std_report_file)

with matplotlib.backends.backend_pdf.PdfPages(avg_std_report_file) as pdf:
    
    jump_corrected_path = os.path.join(parent_folder_data_path , "jump_corrected_files")

    code_set = get_files_with_same_name(jump_corrected_path)

    average_files(successful_list_without_excluded_files, code_set, jump_corrected_path, pdf)

        
        